In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# for the model
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, GRU, RepeatVector, TimeDistributed
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [3]:
# load files from pre-processing

X = np.load('X.npy') #rainpredictions, hour of day, month of year, holiday, ones
y = np.load('y.npy')
TimeHour = pd.read_csv('TimeHour.csv')

In [4]:
# split dataset in test and train

def ReshapeXData(dataframe):
    n_hours = 24
    n_features = dataframe.shape[1]
    days = int(dataframe.shape[0]/24)
    dataframe_shaped = np.reshape(dataframe,(days, n_hours, n_features))
    return dataframe_shaped

def ReshapeYData(dataframe):
    n_hours = 24
    n_features = 1
    days = int(dataframe.shape[0]/24)
    dataframe_shaped = np.reshape(dataframe,(days, n_hours, n_features))
    return dataframe_shaped

print(X.shape)
X_train_gru = X[0:10800,:]
X_test_gru = X[10801:13609,:]
Y_train_gru = y[0:10800]
Y_test_gru = y[10801:13609]

print(Y_train_gru.shape)
X_train_gru_shaped=ReshapeXData(X_train_gru)
X_test_gru_shaped = ReshapeXData(X_test_gru)
Y_train_gru_shaped = ReshapeYData(Y_train_gru)
Y_test_gru_shaped = ReshapeYData(Y_test_gru)
print(Y_train_gru[0:24])
print(Y_train_gru_shaped[0])
print(Y_test_gru[5])

(13612, 39)
(10800,)
[504.06447306 502.99539014 501.56176333 267.72203709 164.6846228
 415.60606761 506.92554056 498.48488889 539.77078944 569.86896236
 573.43357472 554.18056333 519.03118042 518.49464347 518.47689861
 511.91231167 508.04069681 506.03197208 505.73850639 505.39583111
 505.85508306 350.65611313 219.0446198  248.08282937]
[[504.06447306]
 [502.99539014]
 [501.56176333]
 [267.72203709]
 [164.6846228 ]
 [415.60606761]
 [506.92554056]
 [498.48488889]
 [539.77078944]
 [569.86896236]
 [573.43357472]
 [554.18056333]
 [519.03118042]
 [518.49464347]
 [518.47689861]
 [511.91231167]
 [508.04069681]
 [506.03197208]
 [505.73850639]
 [505.39583111]
 [505.85508306]
 [350.65611313]
 [219.0446198 ]
 [248.08282937]]
72.29804405333333


In [5]:
# basic model

from keras.layers import Dense

lr=0.03
epochs=150
batch_size=50
validation_split=0.1

print('Build model...')

n_timesteps, n_features, n_outputs = X_train_gru_shaped.shape[1], X_train_gru_shaped.shape[2], Y_train_gru_shaped.shape[1]
# reshape output into [samples, timesteps, features]

model = Sequential()
model.add(GRU(200, activation='relu', input_shape=(n_timesteps, n_features)))
model.add(RepeatVector(n_outputs))
model.add(GRU(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(100, activation='relu')))
model.add(TimeDistributed(Dense(1)))
model.compile(loss='mae', optimizer='adam')

model.fit(
        X_train_gru_shaped,
        Y_train_gru_shaped,
        epochs=epochs,
        batch_size=batch_size,
        verbose=2,
        validation_split=0.2,
        callbacks=[
            ReduceLROnPlateau(
                verbose=1)])
    
    
#model = Sequential()
#model.add(Dense(1, input_dim=X.shape[1]))
#model.compile(optimizer = optimizers.Adam(lr=lr), loss='mean_squared_error')
#model.fit(X_train, y_train,
#          validation_split=validation_split,
#          epochs=epochs,
#          batch_size=batch_size,
#          shuffle=False)

Build model...
Train on 360 samples, validate on 90 samples
Epoch 1/150
 - 1s - loss: 128.8953 - val_loss: 142.1521
Epoch 2/150
 - 0s - loss: 120.7425 - val_loss: 103.8117
Epoch 3/150
 - 0s - loss: 94.8561 - val_loss: 94.9994
Epoch 4/150
 - 0s - loss: 84.5942 - val_loss: 86.8371
Epoch 5/150
 - 0s - loss: 79.8145 - val_loss: 84.1808
Epoch 6/150
 - 0s - loss: 78.1891 - val_loss: 83.1211
Epoch 7/150
 - 0s - loss: 77.0259 - val_loss: 81.2746
Epoch 8/150
 - 0s - loss: 76.6170 - val_loss: 81.7967
Epoch 9/150
 - 0s - loss: 76.2166 - val_loss: 81.2041
Epoch 10/150
 - 0s - loss: 78.7239 - val_loss: 84.5237
Epoch 11/150
 - 0s - loss: 79.1407 - val_loss: 80.4786
Epoch 12/150
 - 0s - loss: 77.1526 - val_loss: 79.2915
Epoch 13/150
 - 0s - loss: 75.9241 - val_loss: 79.9830
Epoch 14/150
 - 0s - loss: 75.6748 - val_loss: 78.6419
Epoch 15/150
 - 0s - loss: 74.7515 - val_loss: 82.1870
Epoch 16/150
 - 0s - loss: 75.9175 - val_loss: 85.9112
Epoch 17/150
 - 0s - loss: 76.6379 - val_loss: 79.3878
Epoch 18/1

In [6]:
# make a prediction to test how well the model preforms
y_predicted = model.predict(X_test_gru_shaped)
print(y_predicted.shape)
print(Y_test_gru_shaped.shape)
print(y_predicted[1][1])
print(Y_test_gru_shaped[1][1])

(117, 24, 1)
(117, 24, 1)
[131.61899]
[109.39359994]


In [ ]:
### LZS: WE HAVE TO IMPLEMENT HERE SOME METRICS AND A LOT OF PLOTS SO THAT WE CAN SEE HOW WELL THE MODEL PERFORMS
### LZS: AND ALSO TO BE ABLE TO FIND NEW FEATURES TO IMPROVE THE MODEL
### LZS: AND TO EVALUATE IF THE MODEL INDEED IMPROVES WHEN ADDING MORE FEATURES/ OTHER LAYERS

In [7]:
# show the inputs, predicted outputs and real outputs
for i in range(len(X_test_gru_shaped[0])):
        for j in range(len(X_test_gru_shaped[1])):
            print("Predicted=%s, Original=%s" % (y_predicted[i][j], Y_test_gru_shaped[i][j]))

Predicted=[57.22608], Original=[87.90307335]
Predicted=[90.44047], Original=[119.33287184]
Predicted=[93.86158], Original=[70.18270864]
Predicted=[88.79645], Original=[82.55571344]
Predicted=[85.57517], Original=[104.59264897]
Predicted=[84.956924], Original=[72.29804405]
Predicted=[85.389595], Original=[81.48252744]
Predicted=[85.84826], Original=[0.02796992]
Predicted=[86.08742], Original=[0.1253824]
Predicted=[86.21081], Original=[0.3472128]
Predicted=[86.32672], Original=[79.76302907]
Predicted=[86.46804], Original=[80.90010568]
Predicted=[86.62691], Original=[0.08824992]
Predicted=[86.79093], Original=[85.22198364]
Predicted=[86.95452], Original=[139.90204528]
Predicted=[87.11691], Original=[94.68862363]
Predicted=[87.27853], Original=[117.25686593]
Predicted=[87.439384], Original=[90.74470117]
Predicted=[87.599174], Original=[90.482204]
Predicted=[87.7575], Original=[87.95206821]
Predicted=[87.914055], Original=[0.09789472]
Predicted=[88.06866], Original=[83.73856284]
Predicted=[

In [ ]:
# Create a dataframe with a prediction for all datapoints for the csv

y_predicted_all = model.predict(X)
y_pred_all = pd.DataFrame(data=y_predicted_all)

my_df = pd.DataFrame({'Time': TimeHour.loc[:,'TimeHour'].tolist(), 'pred_y': y_pred_all.loc[:,0].tolist()})
my_df.to_csv('FirstPredictions.csv')

In [11]:
df_explore = pd.DataFrame(data=X)
df_explore['TimeHour'] = TimeHour.loc[:, 'TimeHour']

In [ ]:
#date = df_explore.loc[2, "TimeHour"]
#df_explore['TimeHour2'] = pd.to_datetime(df_explore['TimeHour'], format='%Y-%m-%d %H:%M:%S')
#date = df_explore.loc[2, "TimeHour2"]
#type(date)

In [ ]:
#new_date = date + timedelta(hours=1)
#new_date

In [18]:
df_test = pd.DataFrame(data=X)
df_test['TimeHour'] = TimeHour.loc[:, 'TimeHour']

small_df_test = df_explore[545:553]
small_df_test

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,TimeHour
545,0.017310,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 05:00:00
546,0.007029,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 06:00:00
547,0.007944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 07:00:00
548,0.002714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 08:00:00
549,0.006858,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 09:00:00
550,0.136719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 10:00:00
551,0.371582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-25 14:00:00
552,0.091797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-25 15:00:00


In [19]:
small_df_test['TimeHour2'] = 'NA'
small_df_test['TimeHour2'] = pd.to_datetime(df_test['TimeHour'], format='%Y-%m-%d %H:%M:%S')
small_df_test['Diff'] = 'NA'
small_df_test['Before']="NA"

C:\ProgramData\Anaconda3\envs\Keras\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\envs\Keras\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\envs\Keras\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [20]:
# noinspection JupyterKernel
for x in range(546, 553):
    before = small_df_test.loc[x - 1, 'TimeHour2']
    small_df_test.loc[x, 'Before'] = before
    now = small_df_test.loc[x, 'TimeHour2']
    diff = now - before
    small_df_test.loc[x, 'Diff'] = diff
small_df_test

C:\ProgramData\Anaconda3\envs\Keras\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before
545,0.017310,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 05:00:00,2018-01-24 05:00:00,NA,NA
546,0.007029,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 06:00:00,2018-01-24 06:00:00,0 days 01:00:00,2018-01-24 05:00:00
547,0.007944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 07:00:00,2018-01-24 07:00:00,0 days 01:00:00,2018-01-24 06:00:00
548,0.002714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 08:00:00,2018-01-24 08:00:00,0 days 01:00:00,2018-01-24 07:00:00
549,0.006858,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 09:00:00,2018-01-24 09:00:00,0 days 01:00:00,2018-01-24 08:00:00
550,0.136719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 10:00:00,2018-01-24 10:00:00,0 days 01:00:00,2018-01-24 09:00:00
551,0.371582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-25 14:00:00,2018-01-25 14:00:00,1 days 04:00:00,2018-01-24 10:00:00
552,0.091797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-25 15:00:00,2018-01-25 15:00:00,0 days 01:00:00,2018-01-25 14:00:00


In [21]:
small_df_test = small_df_test.drop([545])
small_df_test['diff_hours'] = [x.total_seconds() / 3600 for x in small_df_test['Diff']]

In [22]:
small_df_test = small_df_test.reset_index()
small_df_test = small_df_test.drop(columns=["index"])
small_df_test

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
0,0.007029,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 06:00:00,2018-01-24 06:00:00,0 days 01:00:00,2018-01-24 05:00:00,1.0
1,0.007944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 07:00:00,2018-01-24 07:00:00,0 days 01:00:00,2018-01-24 06:00:00,1.0
2,0.002714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 08:00:00,2018-01-24 08:00:00,0 days 01:00:00,2018-01-24 07:00:00,1.0
3,0.006858,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 09:00:00,2018-01-24 09:00:00,0 days 01:00:00,2018-01-24 08:00:00,1.0
4,0.136719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 10:00:00,2018-01-24 10:00:00,0 days 01:00:00,2018-01-24 09:00:00,1.0
5,0.371582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 14:00:00,2018-01-25 14:00:00,1 days 04:00:00,2018-01-24 10:00:00,28.0
6,0.091797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 15:00:00,2018-01-25 15:00:00,0 days 01:00:00,2018-01-25 14:00:00,1.0


In [23]:
type(small_df_test.loc[1, "diff_hours"])

numpy.float64

In [51]:
def fill_with_average(df, previous, following):
    value_new = list(df.loc[previous])
    value_previous = list(df.loc[previous])
    value_following = list(df.loc[following])
    value_new[0]= (value_previous[0]+value_following[0])/2
    date = df.loc[previous, 'TimeHour2']
    new_date = date + timedelta(hours=1)
    value_new[40]=new_date
    value_new[date.hour]=0.0
    value_new[new_date.hour]=1.0
    #value_new[43]=1.0
    df_length = len(df)
    df.loc[df_length] = value_new
    df = df.sort_values(by= "TimeHour2")
    df = df.reset_index()
    df = df.drop(columns=["index"])
    return df

In [ ]:
#for x, x in small_df_test.iterrows():
#    if small_df_test.loc[x, 'diff_hours'] != 1.0 and small_df_test.loc[x, "diff_hours"] != 24.0:
#        small_df_test.loc[x, 'diff_hours'] = small_df_test.loc[x, 'diff_hours'] - 1
#        small_df_test = fill_with_average(small_df_test, x-1, x)

In [25]:
small_df_test

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
0,0.007029,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 06:00:00,2018-01-24 06:00:00,0 days 01:00:00,2018-01-24 05:00:00,1.0
1,0.007944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 07:00:00,2018-01-24 07:00:00,0 days 01:00:00,2018-01-24 06:00:00,1.0
2,0.002714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 08:00:00,2018-01-24 08:00:00,0 days 01:00:00,2018-01-24 07:00:00,1.0
3,0.006858,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 09:00:00,2018-01-24 09:00:00,0 days 01:00:00,2018-01-24 08:00:00,1.0
4,0.136719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 10:00:00,2018-01-24 10:00:00,0 days 01:00:00,2018-01-24 09:00:00,1.0
5,0.371582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 14:00:00,2018-01-25 14:00:00,1 days 04:00:00,2018-01-24 10:00:00,28.0
6,0.091797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 15:00:00,2018-01-25 15:00:00,0 days 01:00:00,2018-01-25 14:00:00,1.0


In [26]:
x = 0
while x < 11:
    if small_df_test.loc[x, 'diff_hours'] != 1.0 and small_df_test.loc[x, "diff_hours"] != 24.0:
        small_df_test.loc[x, 'diff_hours'] = small_df_test.loc[x, 'diff_hours'] - 1
        small_df_test = fill_with_average(small_df_test, x-1, x)
        x = x + 1
    else:
        x = x + 1

In [27]:
small_df_test

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
0,0.007029,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 06:00:00,2018-01-24 06:00:00,0 days 01:00:00,2018-01-24 05:00:00,1.0
1,0.007944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 07:00:00,2018-01-24 07:00:00,0 days 01:00:00,2018-01-24 06:00:00,1.0
2,0.002714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 08:00:00,2018-01-24 08:00:00,0 days 01:00:00,2018-01-24 07:00:00,1.0
3,0.006858,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 09:00:00,2018-01-24 09:00:00,0 days 01:00:00,2018-01-24 08:00:00,1.0
4,0.136719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 10:00:00,2018-01-24 10:00:00,0 days 01:00:00,2018-01-24 09:00:00,1.0
5,0.254150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 10:00:00,2018-01-24 11:00:00,0 days 01:00:00,2018-01-24 09:00:00,1.0
6,0.312866,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 10:00:00,2018-01-24 12:00:00,0 days 01:00:00,2018-01-24 09:00:00,1.0
7,0.342224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 10:00:00,2018-01-24 13:00:00,0 days 01:00:00,2018-01-24 09:00:00,1.0
8,0.356903,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 10:00:00,2018-01-24 14:00:00,0 days 01:00:00,2018-01-24 09:00:00,1.0
9,0.371582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 14:00:00,2018-01-25 14:00:00,1 days 04:00:00,2018-01-24 10:00:00,24.0


In [ ]:
##Now do this for the whole dataframe

In [28]:
df_test = pd.DataFrame(data=X)
df_test['TimeHour'] = TimeHour.loc[:, 'TimeHour']
df_test

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,TimeHour
0,0.063621,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2018-01-01 12:00:00
1,0.006699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2018-01-01 13:00:00
2,0.004883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2018-01-01 14:00:00
3,0.000977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2018-01-01 15:00:00
4,0.027060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2018-01-01 16:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13607,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2019-08-25 19:00:00
13608,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
13609,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
13610,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN


In [29]:
df_test['TimeHour2'] = 'NA'
df_test['TimeHour2'] = pd.to_datetime(df_test['TimeHour'], format='%Y-%m-%d %H:%M:%S')
df_test['Diff'] = 'NA'
df_test['Before']="NA"
for x in range(1, df_test.shape[0]):
    before = df_test.loc[x - 1, 'TimeHour2']
    df_test.loc[x, 'Before'] = before
    now = df_test.loc[x, 'TimeHour2']
    diff = now - before
    df_test.loc[x, 'Diff'] = diff
df_test

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before
0,0.063621,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,2018-01-01 12:00:00,2018-01-01 12:00:00,NA,NA
1,0.006699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,2018-01-01 13:00:00,2018-01-01 13:00:00,0 days 01:00:00,2018-01-01 12:00:00
2,0.004883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,2018-01-01 14:00:00,2018-01-01 14:00:00,0 days 01:00:00,2018-01-01 13:00:00
3,0.000977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,2018-01-01 15:00:00,2018-01-01 15:00:00,0 days 01:00:00,2018-01-01 14:00:00
4,0.027060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,2018-01-01 16:00:00,2018-01-01 16:00:00,0 days 01:00:00,2018-01-01 15:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13607,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2019-08-25 19:00:00,2019-08-25 19:00:00,0 days 01:00:00,2019-08-25 18:00:00
13608,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaT,NaT,2019-08-25 19:00:00
13609,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaT,NaT,NaT
13610,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaT,NaT,NaT


In [30]:
df_test = df_test.drop(0)

In [31]:
df_test['diff_hours'] = [x.total_seconds() / 3600 for x in df_test['Diff']]

In [ ]:
df_test

In [ ]:
##Delete parts of missing data that are too big and make sure it are still blocks of 24 hours

In [32]:
df_first = df_test[1135:1145]
df_first = df_first.drop([1141,1142])
df_first

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
1136,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-26 05:00:00,2018-02-26 05:00:00,0 days 01:00:00,2018-02-26 04:00:00,1.0
1137,0.013306,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-26 06:00:00,2018-02-26 06:00:00,0 days 01:00:00,2018-02-26 05:00:00,1.0
1138,0.000061,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-26 07:00:00,2018-02-26 07:00:00,0 days 01:00:00,2018-02-26 06:00:00,1.0
1139,0.001282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-26 08:00:00,2018-02-26 08:00:00,0 days 01:00:00,2018-02-26 07:00:00,1.0
1140,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-26 09:00:00,2018-02-26 09:00:00,0 days 01:00:00,2018-02-26 08:00:00,1.0
1143,0.002441,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-28 13:00:00,2018-02-28 13:00:00,0 days 04:00:00,2018-02-28 09:00:00,4.0
1144,0.002441,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-28 14:00:00,2018-02-28 14:00:00,0 days 01:00:00,2018-02-28 13:00:00,1.0
1145,0.000977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-28 15:00:00,2018-02-28 15:00:00,0 days 01:00:00,2018-02-28 14:00:00,1.0


In [33]:
df_test = df_test.drop([1141,1142])
df_test = df_test.reset_index()
df_test = df_test.drop(columns=["index"])
df_test

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
0,0.006699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 13:00:00,2018-01-01 13:00:00,0 days 01:00:00,2018-01-01 12:00:00,1.0
1,0.004883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 14:00:00,2018-01-01 14:00:00,0 days 01:00:00,2018-01-01 13:00:00,1.0
2,0.000977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 15:00:00,2018-01-01 15:00:00,0 days 01:00:00,2018-01-01 14:00:00,1.0
3,0.027060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 16:00:00,2018-01-01 16:00:00,0 days 01:00:00,2018-01-01 15:00:00,1.0
4,0.446828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 17:00:00,2018-01-01 17:00:00,0 days 01:00:00,2018-01-01 16:00:00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13604,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2019-08-25 19:00:00,2019-08-25 19:00:00,0 days 01:00:00,2019-08-25 18:00:00,1.0
13605,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,NaN,NaT,NaT,2019-08-25 19:00:00,NaN
13606,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,NaN,NaT,NaT,NaT,NaN
13607,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,NaN,NaT,NaT,NaT,NaN


In [34]:
df_test[df_test['diff_hours']>1]

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
550,3.715820e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 14:00:00,2018-01-25 14:00:00,1 days 04:00:00,2018-01-24 10:00:00,28.0
668,3.710938e-02,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-31 00:00:00,2018-01-31 00:00:00,0 days 13:00:00,2018-01-30 11:00:00,13.0
670,1.000977e-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-31 14:00:00,2018-01-31 14:00:00,0 days 13:00:00,2018-01-31 01:00:00,13.0
692,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-05 16:00:00,2018-02-05 16:00:00,4 days 05:00:00,2018-02-01 11:00:00,101.0
1067,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-23 09:00:00,2018-02-23 09:00:00,2 days 03:00:00,2018-02-21 06:00:00,51.0
1140,2.441406e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-28 13:00:00,2018-02-28 13:00:00,0 days 04:00:00,2018-02-28 09:00:00,4.0
1509,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-05 20:00:00,2018-04-05 20:00:00,20 days 23:00:00,2018-03-15 21:00:00,503.0
1692,9.765621e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-13 16:00:00,2018-04-13 16:00:00,0 days 06:00:00,2018-04-13 10:00:00,6.0
1805,3.749680e-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-18 17:00:00,2018-04-18 17:00:00,0 days 09:00:00,2018-04-18 08:00:00,9.0
1824,-4.523088e-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-20 12:00:00,2018-04-20 12:00:00,1 days 01:00:00,2018-04-19 11:00:00,25.0


In [35]:
df_error= df_test[650:671]
df_error=df_error.drop([655,656,657,658,659,660,661,662,663,664,665,666,667,668,669])
df_error

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
650,0.284955,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-29 18:00:00,2018-01-29 18:00:00,0 days 01:00:00,2018-01-29 17:00:00,1.0
651,0.081055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-29 19:00:00,2018-01-29 19:00:00,0 days 01:00:00,2018-01-29 18:00:00,1.0
652,0.003906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-29 20:00:00,2018-01-29 20:00:00,0 days 01:00:00,2018-01-29 19:00:00,1.0
653,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-29 21:00:00,2018-01-29 21:00:00,0 days 01:00:00,2018-01-29 20:00:00,1.0
654,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-29 22:00:00,2018-01-29 22:00:00,0 days 01:00:00,2018-01-29 21:00:00,1.0
670,0.010010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-31 14:00:00,2018-01-31 14:00:00,0 days 13:00:00,2018-01-31 01:00:00,13.0


In [36]:
df_test = df_test.drop([655,656,657,658,659,660,661,662,663,664,665,666,667,668,669])
df_test = df_test.reset_index()
df_test = df_test.drop(columns=["index"])
df_test

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
0,0.006699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 13:00:00,2018-01-01 13:00:00,0 days 01:00:00,2018-01-01 12:00:00,1.0
1,0.004883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 14:00:00,2018-01-01 14:00:00,0 days 01:00:00,2018-01-01 13:00:00,1.0
2,0.000977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 15:00:00,2018-01-01 15:00:00,0 days 01:00:00,2018-01-01 14:00:00,1.0
3,0.027060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 16:00:00,2018-01-01 16:00:00,0 days 01:00:00,2018-01-01 15:00:00,1.0
4,0.446828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 17:00:00,2018-01-01 17:00:00,0 days 01:00:00,2018-01-01 16:00:00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13589,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2019-08-25 19:00:00,2019-08-25 19:00:00,0 days 01:00:00,2019-08-25 18:00:00,1.0
13590,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,NaN,NaT,NaT,2019-08-25 19:00:00,NaN
13591,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,NaN,NaT,NaT,NaT,NaN
13592,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,NaN,NaT,NaT,NaT,NaN


In [37]:
df_test[df_test['diff_hours']>1]

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
550,3.715820e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 14:00:00,2018-01-25 14:00:00,1 days 04:00:00,2018-01-24 10:00:00,28.0
655,1.000977e-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-31 14:00:00,2018-01-31 14:00:00,0 days 13:00:00,2018-01-31 01:00:00,13.0
677,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-05 16:00:00,2018-02-05 16:00:00,4 days 05:00:00,2018-02-01 11:00:00,101.0
1052,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-23 09:00:00,2018-02-23 09:00:00,2 days 03:00:00,2018-02-21 06:00:00,51.0
1125,2.441406e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-28 13:00:00,2018-02-28 13:00:00,0 days 04:00:00,2018-02-28 09:00:00,4.0
1494,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-05 20:00:00,2018-04-05 20:00:00,20 days 23:00:00,2018-03-15 21:00:00,503.0
1677,9.765621e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-13 16:00:00,2018-04-13 16:00:00,0 days 06:00:00,2018-04-13 10:00:00,6.0
1790,3.749680e-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-18 17:00:00,2018-04-18 17:00:00,0 days 09:00:00,2018-04-18 08:00:00,9.0
1809,-4.523088e-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-20 12:00:00,2018-04-20 12:00:00,1 days 01:00:00,2018-04-19 11:00:00,25.0
2149,-7.734648e-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-05-04 18:00:00,2018-05-04 18:00:00,0 days 03:00:00,2018-05-04 15:00:00,3.0


In [38]:
df_error= df_test[640:660]
df_error=df_error.drop([641,642,643,644,645,646,647,648,649,650,651,652,653,654,655])
df_error

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
640,0.168701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-29 08:00:00,2018-01-29 08:00:00,0 days 01:00:00,2018-01-29 07:00:00,1.0
656,0.263721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-31 15:00:00,2018-01-31 15:00:00,0 days 01:00:00,2018-01-31 14:00:00,1.0
657,2.194580,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-31 16:00:00,2018-01-31 16:00:00,0 days 01:00:00,2018-01-31 15:00:00,1.0
658,0.023682,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-31 17:00:00,2018-01-31 17:00:00,0 days 01:00:00,2018-01-31 16:00:00,1.0
659,0.115479,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-31 18:00:00,2018-01-31 18:00:00,0 days 01:00:00,2018-01-31 17:00:00,1.0


In [39]:
df_test = df_test.drop([641,642,643,644,645,646,647,648,649,650,651,652,653,654,655])
df_test = df_test.reset_index()
df_test = df_test.drop(columns=["index"])
df_test

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
0,0.006699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 13:00:00,2018-01-01 13:00:00,0 days 01:00:00,2018-01-01 12:00:00,1.0
1,0.004883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 14:00:00,2018-01-01 14:00:00,0 days 01:00:00,2018-01-01 13:00:00,1.0
2,0.000977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 15:00:00,2018-01-01 15:00:00,0 days 01:00:00,2018-01-01 14:00:00,1.0
3,0.027060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 16:00:00,2018-01-01 16:00:00,0 days 01:00:00,2018-01-01 15:00:00,1.0
4,0.446828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 17:00:00,2018-01-01 17:00:00,0 days 01:00:00,2018-01-01 16:00:00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13574,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2019-08-25 19:00:00,2019-08-25 19:00:00,0 days 01:00:00,2019-08-25 18:00:00,1.0
13575,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,NaN,NaT,NaT,2019-08-25 19:00:00,NaN
13576,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,NaN,NaT,NaT,NaT,NaN
13577,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,NaN,NaT,NaT,NaT,NaN


In [40]:
df_test[df_test['diff_hours']>1]

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
550,3.715820e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 14:00:00,2018-01-25 14:00:00,1 days 04:00:00,2018-01-24 10:00:00,28.0
662,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-05 16:00:00,2018-02-05 16:00:00,4 days 05:00:00,2018-02-01 11:00:00,101.0
1037,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-23 09:00:00,2018-02-23 09:00:00,2 days 03:00:00,2018-02-21 06:00:00,51.0
1110,2.441406e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-28 13:00:00,2018-02-28 13:00:00,0 days 04:00:00,2018-02-28 09:00:00,4.0
1479,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-05 20:00:00,2018-04-05 20:00:00,20 days 23:00:00,2018-03-15 21:00:00,503.0
1662,9.765621e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-13 16:00:00,2018-04-13 16:00:00,0 days 06:00:00,2018-04-13 10:00:00,6.0
1775,3.749680e-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-18 17:00:00,2018-04-18 17:00:00,0 days 09:00:00,2018-04-18 08:00:00,9.0
1794,-4.523088e-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-20 12:00:00,2018-04-20 12:00:00,1 days 01:00:00,2018-04-19 11:00:00,25.0
2134,-7.734648e-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-05-04 18:00:00,2018-05-04 18:00:00,0 days 03:00:00,2018-05-04 15:00:00,3.0
3748,0.000000e+00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-07-11 01:00:00,2018-07-11 01:00:00,0 days 02:00:00,2018-07-10 23:00:00,2.0


In [41]:
df_test = df_test.drop([1480])

In [42]:
df_error = df_test[7898:7910]
df_error = df_error.drop([7900,7901,7902,7903,7904,7905])
df_error

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
7899,0.037354,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,2018-12-31 12:00:00,2018-12-31 12:00:00,0 days 01:00:00,2018-12-31 11:00:00,1.0
7906,0.002563,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2019-01-01 13:00:00,2019-01-01 13:00:00,0 days 01:00:00,2019-01-01 12:00:00,1.0
7907,0.100806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2019-01-01 14:00:00,2019-01-01 14:00:00,0 days 01:00:00,2019-01-01 13:00:00,1.0
7908,0.085679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2019-01-01 15:00:00,2019-01-01 15:00:00,0 days 01:00:00,2019-01-01 14:00:00,1.0
7909,0.014871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2019-01-01 16:00:00,2019-01-01 16:00:00,0 days 01:00:00,2019-01-01 15:00:00,1.0
7910,0.034635,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2019-01-01 17:00:00,2019-01-01 17:00:00,0 days 01:00:00,2019-01-01 16:00:00,1.0


In [43]:
df_test = df_test.drop([7900,7901,7902,7903,7904,7905])
df_test = df_test.reset_index()
df_test = df_test.drop(columns=["index"])
df_test

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
0,0.006699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 13:00:00,2018-01-01 13:00:00,0 days 01:00:00,2018-01-01 12:00:00,1.0
1,0.004883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 14:00:00,2018-01-01 14:00:00,0 days 01:00:00,2018-01-01 13:00:00,1.0
2,0.000977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 15:00:00,2018-01-01 15:00:00,0 days 01:00:00,2018-01-01 14:00:00,1.0
3,0.027060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 16:00:00,2018-01-01 16:00:00,0 days 01:00:00,2018-01-01 15:00:00,1.0
4,0.446828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 17:00:00,2018-01-01 17:00:00,0 days 01:00:00,2018-01-01 16:00:00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13567,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2019-08-25 19:00:00,2019-08-25 19:00:00,0 days 01:00:00,2019-08-25 18:00:00,1.0
13568,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,NaN,NaT,NaT,2019-08-25 19:00:00,NaN
13569,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,NaN,NaT,NaT,NaT,NaN
13570,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,NaN,NaT,NaT,NaT,NaN


In [44]:
df_test[df_test['diff_hours']>1]

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
550,3.715820e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 14:00:00,2018-01-25 14:00:00,1 days 04:00:00,2018-01-24 10:00:00,28.0
662,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-05 16:00:00,2018-02-05 16:00:00,4 days 05:00:00,2018-02-01 11:00:00,101.0
1037,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-23 09:00:00,2018-02-23 09:00:00,2 days 03:00:00,2018-02-21 06:00:00,51.0
1110,2.441406e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-28 13:00:00,2018-02-28 13:00:00,0 days 04:00:00,2018-02-28 09:00:00,4.0
1479,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-05 20:00:00,2018-04-05 20:00:00,20 days 23:00:00,2018-03-15 21:00:00,503.0
1661,9.765621e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-13 16:00:00,2018-04-13 16:00:00,0 days 06:00:00,2018-04-13 10:00:00,6.0
1774,3.749680e-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-18 17:00:00,2018-04-18 17:00:00,0 days 09:00:00,2018-04-18 08:00:00,9.0
1793,-4.523088e-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-20 12:00:00,2018-04-20 12:00:00,1 days 01:00:00,2018-04-19 11:00:00,25.0
2133,-7.734648e-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-05-04 18:00:00,2018-05-04 18:00:00,0 days 03:00:00,2018-05-04 15:00:00,3.0
3747,0.000000e+00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-07-11 01:00:00,2018-07-11 01:00:00,0 days 02:00:00,2018-07-10 23:00:00,2.0


In [61]:
df_test.loc[550, 'diff_hours']
df_test

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
0,0.006699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 13:00:00,2018-01-01 13:00:00,0 days 01:00:00,2018-01-01 12:00:00,1.0
1,0.004883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 14:00:00,2018-01-01 14:00:00,0 days 01:00:00,2018-01-01 13:00:00,1.0
2,0.000977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 15:00:00,2018-01-01 15:00:00,0 days 01:00:00,2018-01-01 14:00:00,1.0
3,0.027060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 16:00:00,2018-01-01 16:00:00,0 days 01:00:00,2018-01-01 15:00:00,1.0
4,0.446828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2018-01-01 17:00:00,2018-01-01 17:00:00,0 days 01:00:00,2018-01-01 16:00:00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13634,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2019-08-25 19:00:00,2019-08-25 19:00:00,0 days 01:00:00,2019-08-25 18:00:00,1.0
13635,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,NaN,NaT,NaT,2019-08-25 19:00:00,NaN
13636,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,NaN,NaT,NaT,NaT,NaN
13637,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,NaN,NaT,NaT,NaT,NaN


In [46]:
df_show = df_test[545:560]
df_show

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
545,0.007029,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 06:00:00,2018-01-24 06:00:00,0 days 01:00:00,2018-01-24 05:00:00,1.0
546,0.007944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 07:00:00,2018-01-24 07:00:00,0 days 01:00:00,2018-01-24 06:00:00,1.0
547,0.002714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 08:00:00,2018-01-24 08:00:00,0 days 01:00:00,2018-01-24 07:00:00,1.0
548,0.006858,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 09:00:00,2018-01-24 09:00:00,0 days 01:00:00,2018-01-24 08:00:00,1.0
549,0.136719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 10:00:00,2018-01-24 10:00:00,0 days 01:00:00,2018-01-24 09:00:00,1.0
550,0.371582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 14:00:00,2018-01-25 14:00:00,1 days 04:00:00,2018-01-24 10:00:00,28.0
551,0.091797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 15:00:00,2018-01-25 15:00:00,0 days 01:00:00,2018-01-25 14:00:00,1.0
552,0.061523,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 16:00:00,2018-01-25 16:00:00,0 days 01:00:00,2018-01-25 15:00:00,1.0
553,0.293457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 17:00:00,2018-01-25 17:00:00,0 days 01:00:00,2018-01-25 16:00:00,1.0
554,0.000488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 18:00:00,2018-01-25 18:00:00,0 days 01:00:00,2018-01-25 17:00:00,1.0


In [58]:
x = 0
while x < 13000:
    if df_test.loc[x, 'diff_hours'] != 1.0 and df_test.loc[x, "diff_hours"] != 24.0 and df_test.loc[x, 'diff_hours'] != 96.0 and df_test.loc[x, 'diff_hours'] != 48.0 and df_test.loc[x, 'diff_hours'] != 480.0:
        print(x)
        df_test.loc[x, 'diff_hours'] = df_test.loc[x, 'diff_hours'] - 1
        df_test = fill_with_average(df_test, x-1, x)
        x = x - 1
    else:
        x = x + 1

print ('Missing values approximated')

Missing values approximated


In [60]:
#df_test[df_test['diff_hours']>1]
df_test[549:560]

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
549,0.136719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 10:00:00,2018-01-24 10:00:00,0 days 01:00:00,2018-01-24 09:00:00,1.0
550,0.254150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 10:00:00,2018-01-24 11:00:00,0 days 01:00:00,2018-01-24 09:00:00,1.0
551,0.312866,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 10:00:00,2018-01-24 12:00:00,0 days 01:00:00,2018-01-24 09:00:00,1.0
552,0.371582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 14:00:00,2018-01-25 14:00:00,1 days 04:00:00,2018-01-24 10:00:00,24.0
553,0.091797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 15:00:00,2018-01-25 15:00:00,0 days 01:00:00,2018-01-25 14:00:00,1.0
554,0.061523,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 16:00:00,2018-01-25 16:00:00,0 days 01:00:00,2018-01-25 15:00:00,1.0
555,0.293457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 17:00:00,2018-01-25 17:00:00,0 days 01:00:00,2018-01-25 16:00:00,1.0
556,0.000488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 18:00:00,2018-01-25 18:00:00,0 days 01:00:00,2018-01-25 17:00:00,1.0
557,0.001465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 19:00:00,2018-01-25 19:00:00,0 days 01:00:00,2018-01-25 18:00:00,1.0
558,0.089844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 20:00:00,2018-01-25 20:00:00,0 days 01:00:00,2018-01-25 19:00:00,1.0
